# 필요한 라이브러리 임포트

In [1]:
import requests
import pandas as pd
from tqdm import tqdm  # 진행률 표시용
from datetime import datetime, timedelta

# 날짜 리스트 생성

In [2]:
start_date = datetime(2022, 1, 1)
end_date = datetime(2025, 10, 10)

date_list = [(start_date + timedelta(days=x)).strftime('%Y%m%d') 
             for x in range((end_date - start_date).days + 1)]

# 하루씩 데이터를 요청하고 합치기

In [3]:
all_data = []

url = "https://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020505",
    "X-Requested-With": "XMLHttpRequest",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8"
}

for date in tqdm(date_list):
    params = {
        "bld": "dbms/MDC/STAT/standard/MDCSTAT03901",
        "mktId": "STK",  # 코스피
        "trdDd": date,
        "share": "1",
        "money": "1",
        "csvxls_isNo": "false"
    }
    response = requests.post(url, data=params, headers=headers)
    data = response.json()
    
    temp_df = pd.DataFrame(data['block1'])
    
    # '-' 포함된 행 제거
    temp_df = temp_df[~temp_df['MKTCAP'].str.contains('-')]
    
    # 행이 남아 있는 경우만 처리
    if not temp_df.empty:
        temp_df['MKTCAP'] = temp_df['MKTCAP'].str.replace(',', '').astype('Int64')
        temp_df['DATE'] = date
        all_data.append(temp_df)

# 모든 데이터를 하나의 DataFrame으로 합치기
full_df = pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()

100%|██████████| 1379/1379 [09:05<00:00,  2.53it/s]


# 업종별 합계 및 정렬

In [4]:
if not full_df.empty:
    sector_df = full_df.groupby(['DATE', 'IDX_IND_NM'])['MKTCAP'].sum().reset_index()
    sector_df = sector_df.sort_values(['DATE', 'MKTCAP'], ascending=[True, False])
else:
    pass

# 엑셀로 저장

In [ ]:
file_path = "../data/KRX_sector_mktcap.xlsx"

try:
    existing_df = pd.read_excel(file_path)
except FileNotFoundError:
    existing_df = pd.DataFrame()  # 파일이 없으면 빈 DataFrame 생성

# 새로운 데이터 sector_df와 합치기
combined_df = pd.concat([existing_df, sector_df], ignore_index=True)

# 다시 엑셀로 저장
combined_df.to_excel(file_path, index=False)


UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 15-16: invalid continuation byte